In [ ]:
# load TensorFlow
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# import other modules
import beatmup
import beatmup_keras
import numpy
import os
import sys
from model import make_model
from imagenet_tools import ImageSet

# Set global configuration

# path to ImageNet 2012 dataset
imagenet_path = '/imagenet'

# folder to store checkpoints during training
checkpoints_dir = 'checkpoints'

# classes numbers: 119 dogs and tiger cat
classes = list(range(151,269)) + [275, 282]

In [ ]:
# Setting up the model
model = make_model(input_size=385, activation=beatmup_keras.sigmoid_like, num_classes=120)
model.summary()

In [ ]:
# Validation set loading
image_size = model.layers[0].input_shape[0][1]
val_set = ImageSet(os.path.join(imagenet_path, 'Annotations', 'CLS-LOC', 'val'),
                   os.path.join(imagenet_path, 'Data', 'CLS-LOC', 'val'),
                   os.path.join(imagenet_path, 'val_cache.txt'),
                   batch_size=1,
                   image_size=image_size,
                   use_annotations=False)
val_set.supply_class_names('map_clsloc.txt')
val_set.filter(classes)

In [ ]:
# Checkpoint loading
from glob import glob
checkpoint_files = sorted(glob(os.path.join(checkpoints_dir, '*.h5')))

if not checkpoint_files:
    raise Exception("No checkpoint found in '%s'" % (checkpoints_dir))
model.load_weights(checkpoint_files[-1])
print('Loading %s' % (checkpoint_files[-1]))

In [ ]:
# Run validation pass
model.evaluate(val_set)

In [ ]:
# Beatmup model export

# Convert the model into Beatmup model
ctx = beatmup.Context()
beatmup_model, beatmup_model_data = beatmup_keras.export_model(model, ctx)

# Add image sampler at the beginning
image_size = model.layers[0].input_shape[0][1]
tail = beatmup_model.get_first_operation()
sampler = beatmup.nnets.ImageSampler("input", (image_size, image_size))
beatmup_model.add_operation(tail.name, sampler)
beatmup_model.add_connection(sampler.name, tail.name)

# Add softmax at the end
beatmup_model.append(beatmup.nnets.Softmax("softmax"), True)

# Serialize the model, put the model into "" chunk (sort of convention)
code = beatmup_model.serialize()
beatmup_model_data[""] = code.encode("ascii")

# Add class names
beatmup_model_data["labels"] = "".join(val_set.class_names).encode("ascii")

# Save to a chunkfile
beatmup_model_data.save("dog_classifier.chunks", False)

print('All good.')

In [ ]:
# Run validation pass with Beatmup model
inference = beatmup.nnets.InferenceTask(beatmup_model, beatmup_model_data)
input_op = beatmup_model.get_first_operation()
softmax_op = beatmup_model.get_last_operation()

score = 0
for i in range(len(val_set)):
    # Get image from training set
    image, labels = val_set[i]
    image = beatmup.Bitmap(ctx, image)

    # Connect image to the inference task
    inference.connect(image, input_op)
    
    # Run inference
    ctx.perform_task(inference)
    
    # Get predicted probabilities
    probabilities = softmax_op.get_probabilities()
    
    # Get the class label
    prediction = numpy.argmax(probabilities)
    
    # Compare with the ground truth
    ground_truth = numpy.argmax(labels)
    if prediction == ground_truth:
        score += 1

    # Print some info
    num_images = i + 1
    acc  = 100 * score / num_images
    print('\r %d / %d images, accuracy: %0.2f%%' % (num_images, len(val_set), acc), end='')
    
    # Collect garbage
    ctx.empty_gpu_recycle_bin()